In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/*Utad/3ºAño/IA/Practicas_IA/Practica_2/intervenciones.zip

Archive:  /content/drive/MyDrive/*Utad/3ºAño/IA/Practicas_IA/Practica_2/intervenciones.zip
  inflating: intervencionesAbascal.txt  
  inflating: intervencionesCasado.txt  
  inflating: intervencionesSanchez.txt  


# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [3]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os


## Lectura de ficheros de datos

In [4]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Read the data

First, look in the text:

In [5]:
# Read, then decode for py2 compat.
text = open(datos_casado, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 105940 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima proponen las mismas recetas fracasadas que nos llevaron a la peor crisis económica de nuestra historia: más despilfarro, más déficit y más impuestos. Pero 


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

92 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[47, 48, 49, 50, 51, 52, 53], [70, 71, 72]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [11]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


# Fases propuestas para la elaboración del modelo

### 1. Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(105940,), dtype=int64, numpy=array([38, 51, 81, ..., 47, 20,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

S
e
ñ
o
r
 
S
á
n
c


In [18]:
seq_length = 100

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'S' b'e' b'\xc3\xb1' b'o' b'r' b' ' b'S' b'\xc3\xa1' b'n' b'c' b'h' b'e'
 b'z' b',' b' ' b's' b'u' b's' b' ' b'r' b'e' b'c' b'e' b't' b'a' b's'
 b' ' b'e' b'c' b'o' b'n' b'\xc3\xb3' b'm' b'i' b'c' b'a' b's' b' ' b's'
 b'o' b'n' b' ' b't' b'a' b'n' b' ' b'c' b'r' b'e' b'\xc3\xad' b'b' b'l'
 b'e' b's' b' ' b'c' b'o' b'm' b'o' b' ' b's' b'u' b's' b' ' b'p' b'r'
 b'o' b'm' b'e' b's' b'a' b's' b' ' b'e' b'l' b'e' b'c' b't' b'o' b'r'
 b'a' b'l' b'e' b's' b',' b' ' b'y' b' ' b'e' b'n' b'c' b'i' b'm' b'a'
 b' ' b'p' b'r' b'o' b'p' b'o' b'n'], shape=(101,), dtype=string)


In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Se\xc3\xb1or S\xc3\xa1nchez, sus recetas econ\xc3\xb3micas son tan cre\xc3\xadbles como sus promesas electorales, y encima propon'
b'en las mismas recetas fracasadas que nos llevaron a la peor crisis econ\xc3\xb3mica de nuestra historia: m\xc3\xa1s'
b' despilfarro, m\xc3\xa1s d\xc3\xa9ficit y m\xc3\xa1s impuestos. Pero el Partido Popular es un partido de Estado y tambi\xc3\xa9n '
b'de Gobierno, aunque estemos temporalmente en la oposici\xc3\xb3n. Por eso el lunes le ofrec\xc3\xad pactar los Pres'
b'upuestos Generales si rompe con los independentistas, una oferta, por cierto, a la que usted no ha co'


In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Se\xc3\xb1or S\xc3\xa1nchez, sus recetas econ\xc3\xb3micas son tan cre\xc3\xadbles como sus promesas electorales, y encima propo'
Target: b'e\xc3\xb1or S\xc3\xa1nchez, sus recetas econ\xc3\xb3micas son tan cre\xc3\xadbles como sus promesas electorales, y encima propon'


### 2. Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [25]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 3. Build The Model

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

In [26]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 93) # (batch_size, sequence_length, vocab_size)


## 4. Summary y fit del modelo



Aconsejable el uso de GPU en Google Colab, aunque yo he conseguido hacer 200 épocas de mi modelo en un tiempo razonable (menos de 1 hora).

**RECUERDA GRABAR EL MODELO ENTRENADO PARA PODER REUTILIZARLO POSTERIORMENTE**

In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23808     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  95325     
                                                                 
Total params: 4057437 (15.48 MB)
Trainable params: 4057437 (15.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [32]:
sampled_indices

array([28, 77, 61, 69, 16, 38, 68, 13, 57,  0,  0, 33, 57, 48, 31, 88,  5,
       67, 45, 13, 71, 47,  1, 33, 49, 88, 63, 88, 72, 23, 28, 52, 51, 50,
       61, 52, 12, 53, 16, 50, 39, 88, 75, 28, 23, 21, 68, 46, 83, 36,  0,
        5, 31,  1, 54, 77, 85, 80, 45, 34, 10, 78, 42, 60, 38, 21,  5, 45,
       15, 12,  8, 50, 76, 56, 79, 27, 67, 70, 87, 36, 11, 56, 44, 85, 16,
       53, 91, 13, 74, 58, 20, 33,  3,  2, 24, 81,  6, 87, 17, 33])

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ed ha sido el convidado de piedra en una negociaci\xc3\xb3n en la que ha tenido el curioso m\xc3\xa9todo de la esc'

Next Char Predictions:
 b'H\xc3\x81ow8Sv5k[UNK][UNK]NkbL\xe2\x80\x94,u[5ya\nNc\xe2\x80\x94q\xe2\x80\x94zCHfedof4g8dT\xe2\x80\x94\xc2\xbbHCAv]\xc3\xb6Q[UNK],L\nh\xc3\x81\xc3\xbc\xc3\xad[O2\xc3\xa1XnSA,[740d\xc2\xbfj\xc3\xa9Gux\xe2\x80\x8cQ3jZ\xc3\xbc8g\xe2\x80\x995\xc2\xabl?N! D\xc3\xb1-\xe2\x80\x8c9N'


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.53477, shape=(), dtype=float32)


In [36]:
tf.exp(example_batch_mean_loss).numpy()

93.20207

In [37]:
model.compile(optimizer='adam', loss=loss)

In [38]:

'''
import shutil
import os

# Directorio que acabamos de crear
directory_to_delete = './training_checkpoints'

# Eliminar el directorio
shutil.rmtree(directory_to_delete)
'''

"\nimport shutil\nimport os\n\n# Directorio que acabamos de crear\ndirectory_to_delete = './training_checkpoints'\n\n# Eliminar el directorio\nshutil.rmtree(directory_to_delete)\n"

In [39]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    period=10)

In [40]:
EPOCHS = 400

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/400
16/16 [==============================] - 4s 86ms/step - loss: 4.2255
Epoch 2/400
16/16 [==============================] - 1s 53ms/step - loss: 3.1395
Epoch 3/400
16/16 [==============================] - 1s 57ms/step - loss: 2.8119
Epoch 4/400
16/16 [==============================] - 1s 54ms/step - loss: 2.5571
Epoch 5/400
16/16 [==============================] - 1s 51ms/step - loss: 2.3932
Epoch 6/400
16/16 [==============================] - 1s 51ms/step - loss: 2.2994
Epoch 7/400
16/16 [==============================] - 1s 51ms/step - loss: 2.2327
Epoch 8/400
16/16 [==============================] - 1s 62ms/step - loss: 2.1788
Epoch 9/400
16/16 [==============================] - 1s 61ms/step - loss: 2.1304
Epoch 10/400
16/16 [==============================] - 1s 65ms/step - loss: 2.0843
Epoch 11/400
16/16 [==============================] - 1s 53ms/step - loss: 2.0358
Epoch 12/400
16/16 [==============================] - 1s 52ms/step - loss: 1.9879
Epoch 13/400
16/16 [=====

In [42]:
# Acceder a las métricas de entrenamiento
train_loss = history.history['loss']

# Encontrar la época con la pérdida más baja
min_loss_epoch = train_loss.index(min(train_loss)) + 1  # Sumar 1 porque las épocas comienzan desde 1

# Imprimir la época con la pérdida más baja
print(f"Época con la pérdida más baja: {min_loss_epoch}, Loss: {min(train_loss)}")

Época con la pérdida más baja: 378, Loss: 0.043975211679935455


## 5. Genera texto y evalúa su calidad

In [43]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [44]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [45]:
start = time.time()
states = None
next_char = tf.constant(['Política'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Política. Pero el 4 % se mantuvo. Por tanto, hay que hacer reformas estructurales, no dilapidar todos los pactos propuestos. Es unos mintos como la ley de Salud Pública del mismo año establece en su artículo 14 la competencia automática del Ministerio de Sanidad en tres años de gobierno. Hay que poner en marcha un nuevo, y en su artículo 30 contempla el mando único de gestión de crisis en situación de emergencia puedan encontrar a los desaparecidos. Señor Sánchez, le tengo que enterar a través de fuentes comunitarias de que usted incluye con su astíando con ruedas de molino. No llame diálogo a lo que es una humillación. Todo empezó con el Tinell y el respoto a la señora Mertxe Aizpurud para el año 2019  y no se pudo centarlo que su gestión ha sido un fracaso y por eso tenemos la marca de la experiencia y un presidente del Gobierno de la cuando la ventado que ya se puede pactar y blanquear para que decrete luto nacional, para que cambie el condenado inhabilitado y con un imputado por or

In [46]:
#!zip -r /content/abascal.zip /content/training_checkpoints

## 6. Trabajo adicional

Por ejempl, poner en cadena los tres modelos para que "dialoguen" entre sí

In [47]:
tf.saved_model.save(one_step_model, 'one_step_casado')


No he encontrado la lanera de cargar un modelo a partir de un checkpoint